In [7]:
import os
from pyprojroot import here
import pandas as pd
import chromadb
from openai import AzureOpenAI
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
print(load_dotenv()) 
from openai import OpenAI
import openai

True


In [3]:
import os
from langchain.chat_models import ChatOpenAI

openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name="gpt-3.5-turbo",  
    temperature=0.0
)

chroma_client = chromadb.PersistentClient(path=str(here("data/chroma")))

/tmp/ipykernel_2399075/1124974890.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [4]:
########  Create a collection for data injection 

## throws an error if the table already exists 


collection = chroma_client.create_collection(name="titanic_small")

In [6]:
file_dir = here("data/for_upload/titanic_small.csv")
df = pd.read_csv(file_dir,nrows=5)
df

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35,0,0,8.0500


In [9]:
#######    Each row gonna be a single chunk in the vector db    ################ 


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Lists to store results
docs = []
metadatas = []
ids = []
embeddings = []

# Loop over rows in your DataFrame (assumes df is already defined)
for index, row in df.iterrows():
    output_str = ""
    for col in df.columns:
        output_str += f"{col}: {row[col]},\n"
    
    # New embedding API usage
    response = client.embeddings.create(
        input=output_str,
        model="text-embedding-ada-002"
    )
    
    embedding = response.data[0].embedding
    embeddings.append(embedding)
    docs.append(output_str)
    metadatas.append({"source": "titanic_small"})
    ids.append(f"id{index}")

In [10]:
docs 

['Survived: 0,\nPclass: 3,\nName: Mr. Owen Harris Braund,\nSex: male,\nAge: 22,\nSiblings/Spouses Aboard: 1,\nParents/Children Aboard: 0,\nFare: 7.25,\n',
 'Survived: 1,\nPclass: 1,\nName: Mrs. John Bradley (Florence Briggs Thayer) Cumings,\nSex: female,\nAge: 38,\nSiblings/Spouses Aboard: 1,\nParents/Children Aboard: 0,\nFare: 71.2833,\n',
 'Survived: 1,\nPclass: 3,\nName: Miss. Laina Heikkinen,\nSex: female,\nAge: 26,\nSiblings/Spouses Aboard: 0,\nParents/Children Aboard: 0,\nFare: 7.925,\n',
 'Survived: 1,\nPclass: 1,\nName: Mrs. Jacques Heath (Lily May Peel) Futrelle,\nSex: female,\nAge: 35,\nSiblings/Spouses Aboard: 1,\nParents/Children Aboard: 0,\nFare: 53.1,\n',
 'Survived: 0,\nPclass: 3,\nName: Mr. William Henry Allen,\nSex: male,\nAge: 35,\nSiblings/Spouses Aboard: 0,\nParents/Children Aboard: 0,\nFare: 8.05,\n']

In [11]:
print(metadatas)
print(ids)

[{'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}]
['id0', 'id1', 'id2', 'id3', 'id4']


In [12]:
embeddings[0][:10]

[-0.0055847191251814365,
 -0.01992053911089897,
 -0.017392899841070175,
 -0.025085123255848885,
 0.010349662974476814,
 0.03464916720986366,
 -0.013396494090557098,
 -0.0008295954903587699,
 -0.02501680888235569,
 -0.017502201721072197]

In [13]:
collection.add(
    documents=docs,
    metadatas=metadatas,
    embeddings=embeddings,
    ids=ids
)


In [14]:
print("Number of vectors in vectordb:", collection.count())

Number of vectors in vectordb: 5


In [ ]:
###################################  RAG  ########################################


query_texts = "what's the average age of survivors"
response = client.embeddings.create(
        input = query_texts,
        model= "text-embedding-ada-002"
    )
query_embeddings = response.data[0].embedding



In [17]:
#############  Load the chromaDB collection for vector search  ############ 

vectordb = chroma_client.get_collection(name="titanic_small")
vectordb.count()



5

In [18]:
results = vectordb.query(
    query_embeddings = query_embeddings,
    n_results=1 #top_k
)

results



{'ids': [['id4']],
 'embeddings': None,
 'documents': [['Survived: 0,\nPclass: 3,\nName: Mr. William Henry Allen,\nSex: male,\nAge: 35,\nSiblings/Spouses Aboard: 0,\nParents/Children Aboard: 0,\nFare: 8.05,\n']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'titanic_small'}]],
 'distances': [[0.4858440373100639]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [19]:
system_role = "You will recieve the user's question along with the search results of that question over a database. Give the user the proper answer."
prompt = f"User's question: {query_texts} \n\n Search results:\n {results}"

messages = [
    {"role": "system", "content": str(
        system_role
        )},
    {"role": "user", "content": prompt}
]

In [22]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # or "gpt-4"
    messages=messages
)


In [23]:
response.choices[0].message.content

'Based on the search result, the average age of survivors on the Titanic is not provided. However, the information for one passenger, Mr. William Henry Allen, shows that he was 35 years old. Further analysis of the dataset would be needed to calculate the average age of survivors.'

In [24]:
df

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35,0,0,8.0500
